In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
file_path = "/content/drive/MyDrive/KUBIG-DLcontest/dataset/개정 후 전처리 결과물 (08 26)/"

X_train = np.load(file_path + 'X_train.npy')
X_test = np.load(file_path + 'X_test.npy')
y_train = np.load(file_path + 'Y_train.npy')


In [ ]:
X_train[0]

In [ ]:
max_len = 15
vocab_size = 26124
embedding_dim = 128 # vocab_size보다 훨씬 작게 설정

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length = max_len, trainable = True))
model.add(Bidirectional(LSTM(64, dropout = 0.2, recurrent_dropout = 0.2, return_sequences = True)))
model.add(Bidirectional(LSTM(64, dropout = 0.2, recurrent_dropout = 0.2, return_sequences = True)))
model.add(Bidirectional(LSTM(64, dropout = 0.2, recurrent_dropout = 0.2)))
model.add(Dense(7, activation = "softmax"))


In [ ]:
model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.3, random_state = 1000)
model.compile(loss= SparseCategoricalCrossentropy(), optimizer='adam', metrics=['acc'])
history = model.fit(X_train, y_train,
                    batch_size=128, epochs=30,
                    callbacks=[es, mc], validation_data=(X_valid, y_valid), validation_split = 0.3)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
pred_1 = []
for p in predictions:
    pred_1.append(np.argmax(p))

In [ ]:
sample_sub = pd.read_csv("/content/drive/MyDrive/KUBIG-DLcontest/dataset/sample_submission.csv")
sample_sub['topic_idx'] = pred_1
sample_sub.to_csv("sample_sub.csv", index = False)


In [ ]:
sample_sub